In [ ]:
# Import required modules
import time
import pandas as pd
from marketwatch import MarketWatch

# Initialize MarketWatch API
username = 'your_username_here'  # Replace with your username
password = 'your_password_here'  # Replace with your password
game_name = 'your_game_name_here'  # Replace with your game name

# Initialize MarketWatch
marketwatch = MarketWatch(username, password)

# Function to calculate EMA
def calculate_ema(price_series, periods):
    return price_series.ewm(span=periods, adjust=False).mean()

# Function to trade based on EMA crossover and check if the trade worked
def trade_on_crossover(df, game_name):
    short_term = df['EMA_20'].iloc[-1]
    long_term = df['EMA_80'].iloc[-1]
    prev_short_term = df['EMA_20'].iloc[-2]
    prev_long_term = df['EMA_80'].iloc[-2]
    
    if short_term > long_term and prev_short_term <= prev_long_term:
        print('Golden Cross detected. Attempting to buy stock.')
        marketwatch.buy(game_name, 'AAPL', 100)
        
        # Check if the buy order was successful
        positions = marketwatch.get_positions(game_name)
        if any(pos['ticker'] == 'AAPL' for pos in positions):
            print('Buy order successful.')
        else:
            print('Buy order failed.')
        
    elif short_term < long_term and prev_short_term >= prev_long_term:
        print('Death Cross detected. Attempting to sell stock.')
        marketwatch.sell(game_name, 'AAPL', 100)
        
        # Check if the sell order was successful
        positions = marketwatch.get_positions(game_name)
        if not any(pos['ticker'] == 'AAPL' for pos in positions):
            print('Sell order successful.')
        else:
            print('Sell order failed.')

In [ ]:
# Collect historical data by calling get_price() at intervals
price_data = []
for _ in range(160):  # Collect more data points for initial EMA calculation
    price = marketwatch.get_price('AAPL')
    price_data.append(price)
    if len(price_data) > 150:  # Keep the data window at 150 points
        price_data.pop(0)
    
    # Convert the price data to a Pandas DataFrame
    df = pd.DataFrame(price_data, columns=['Close'])
    
    # Calculate EMA for 20 and 80 periods
    for period in [20, 80]:
        ema_column_name = f'EMA_{period}'
        df[ema_column_name] = calculate_ema(df['Close'], period)
    
    # Trade based on EMA crossover if we have enough data for EMA 80
    if len(df) >= 80:
        trade_on_crossover(df, game_name)
    
    time.sleep(60)  # Wait for 60 seconds before the next call